In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  # default is ‘last_expr'

%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/Users/siyuyang/Source/repos/GitHub_MSFT/CameraTraps')

In [32]:
import json
import os
from collections import Counter, defaultdict
from random import sample
import math

from data_management.megadb.schema import sequences_schema_check
from data_management.annotations.add_bounding_boxes_to_megadb import *
from data_management.megadb.converters.cct_to_megadb import make_cct_embedded, process_sequences, write_json

In [4]:
# annotations

anno_batch_10 = '/Users/siyuyang/Source/temp_data/CameraTrap/annotations/201910/batch10'
anno_batch_11 = '/Users/siyuyang/Source/temp_data/CameraTrap/annotations/201910/MsWLB_imerit_batch_11_18sep2019'

For some (especially the first few) datasets, the sections "Make into MegaDB format" and "Add bounding boxes" are run after loading the database and spot checking.

## awc_190430

In [4]:
with open('/Users/siyuyang/OneDrive - Microsoft/AI4Earth/CameraTrap/Databases/cosmos/awc_190430_20190816_embedded.json') as f:
    awc_embedded = json.load(f)

In [5]:
len(awc_embedded)

159273

In [6]:
awc_embedded[100]

{'width': 2048,
 'file_name': 'WetTropics/2018/Spurgeon Bettong/QWT-S-TSCAM087 (WF269-269)/IMG_0805.JPG',
 'image_id': '102328',
 'dataset': 'awc_190430',
 'height': 1536,
 'location': 'QWT-S-TSCAM087 (WF269-269)',
 'annotations': {'species': ['empty']}}

In [8]:
awc_locs = Counter()

for i in awc_embedded:
    awc_locs.update([i['location']])
    
len(awc_locs)

193

In [ ]:
# some species arrays had two "empty" labels

In [9]:
for e in awc_embedded:
    if 'annotations' in e:
        e['annotations']['species'] = list(set(e['annotations']['species']))

## bellevue_190602

In [26]:
with open('/Users/siyuyang/OneDrive - Microsoft/AI4Earth/CameraTrap/Databases/cosmos/bellevue_190602_20190812_embedded.json') as f:
    bellevue_embedded = json.load(f)

In [27]:
len(bellevue_embedded)

1848

In [28]:
bellevue_embedded[100]

{'image_id': 'd3695634-a9a9-11e9-a392-5cf370671a19',
 'file_name': '2019.01.20/newcam/empty/DSCF0057.JPG',
 'annotations': {'species': ['empty']},
 'dataset': 'bellevue_190602'}

## idfg

In [5]:
with open('/Users/siyuyang/OneDrive - Microsoft/AI4Earth/CameraTrap/Databases/cosmos/idfg_20190807_embedded.json') as f:
    idfg_embedded = json.load(f)

In [6]:
len(idfg_embedded)

693870

In [7]:
sample(idfg_embedded, 1)

[{'image_id': 'St.Joe_elk~AM129~Trip 1~100RECNX~RCNX0302',
  'file_name': 'St.Joe_elk/AM129/Trip 1/100RECNX/RCNX0302.JPG',
  'dataset': 'idfg',
  'annotations': {'species': ['deer']},
  'datetime': '07-Jan-2016 08:38:34',
  'location': 'St_Joe_elk+AM129'}]

There are bboxes

In [16]:
# some images (dummy sequences) do not have the class label...

for e in idfg_embedded:
    if 'annotations' not in e or 'species' not in e['annotations']:
        e['annotations'] = {
            'species': ['__label_unavailable']
        }

In [36]:
# some images from Trip 2 of a location which we got vehicle annotations for were not in the CCT
# it seems, so adding them here manually:

idfg_missing

['Beaverhead_elk~AM25~Trip 2~100RECNX~2016-03-12 15-01-50 M 5_5.JPG',
 'Beaverhead_elk~AM25~Trip 2~100RECNX~2016-02-19 09-31-54 M 3_5.JPG',
 'Beaverhead_elk~AM25~Trip 2~100RECNX~2016-02-19 09-31-55 M 4_5.JPG',
 'Beaverhead_elk~AM25~Trip 2~100RECNX~2016-03-14 12-40-00 T.JPG',
 'Beaverhead_elk~AM25~Trip 2~100RECNX~2016-03-12 15-01-47 M 2_5.JPG',
 'Beaverhead_elk~AM25~Trip 2~100RECNX~2016-02-19 09-31-53 M 1_5.JPG',
 'Beaverhead_elk~AM25~Trip 2~100RECNX~2016-02-19 09-31-56 M 5_5.JPG',
 'Beaverhead_elk~AM25~Trip 2~100RECNX~2016-03-12 15-01-52 M 2_5.JPG',
 'Beaverhead_elk~AM25~Trip 2~100RECNX~2016-03-12 15-01-48 M 3_5.JPG',
 'Beaverhead_elk~AM25~Trip 2~100RECNX~2016-03-12 15-01-51 M 1_5.JPG',
 'Beaverhead_elk~AM25~Trip 2~100RECNX~2016-03-14 12-40-00 M 3_5.JPG',
 'Beaverhead_elk~AM25~Trip 2~100RECNX~2016-03-12 15-01-46 M 1_5.JPG',
 'Beaverhead_elk~AM25~Trip 2~100RECNX~2016-03-12 15-01-49 M 4_5.JPG',
 'Beaverhead_elk~AM25~Trip 2~100RECNX~2016-03-14 12-39-59 M 2_5.JPG',
 'Beaverhead_elk~AM25~Tr

In [38]:
idfg_missing_items = []
for i in idfg_missing:
    item = {
        'image_id': i.split('.JPG')[0],
      'file_name': i.replace('~', '/'),
      'dataset': 'idfg',
      'annotations': {'species': ['__label_unavailable']},
      'location': 'Beaverhead_elk+AM25'
    }
    idfg_missing_items.append(item)

In [40]:
len(idfg_embedded)
idfg_embedded.extend(idfg_missing_items)
len(idfg_embedded)

693870

693886

## nacti

In [5]:
with open('/Users/siyuyang/OneDrive - Microsoft/AI4Earth/CameraTrap/Databases/cosmos/nacti_20190819_embedded.json') as f:
    nacti_embedded = json.load(f)

In [6]:
len(nacti_embedded)

3382215

In [7]:
sample(nacti_embedded, 2)

[{'height': 1536,
  'width': 2048,
  'annotations': {'species': ['domestic cow']},
  'image_id': 'FL-07_06_29_2015_FL-07_0042512',
  'location': 'Archbold, FL',
  'study': 'FL',
  'dataset': 'nacti',
  'file_name': 'part1/sub147/FL-07_06_29_2015_FL-07_0042512.jpg'},
 {'height': 1536,
  'width': 2048,
  'annotations': {'species': ['red deer']},
  'image_id': '2014_Unit70_Ivan053_img0337',
  'location': 'San Juan Mntns, Colorado',
  'study': 'CPW',
  'dataset': 'nacti',
  'file_name': 'part0/sub015/2014_Unit70_Ivan053_img0337.jpg'}]

In [25]:
species_counter = Counter()
for i in nacti_embedded:
    species_counter.update(i['annotations']['species'])

In [8]:
nacti_locs = Counter()

for i in nacti_embedded:
    nacti_locs.update([i['location']])
    
len(nacti_locs)

4

In [9]:
nacti_locs

Counter({'Archbold, FL': 1802622,
         'Lebec, California': 780549,
         '': 469356,
         'San Juan Mntns, Colorado': 329688})

In [30]:
# Counter({'CA': 261600, 'FL': 207756})

for i in nacti_embedded:
    if i['location'] == '':
        i['location'] = i['image_id'].split('-')[0]

## rspb_gola_20190409

In [25]:
with open('/Users/siyuyang/OneDrive - Microsoft/AI4Earth/CameraTrap/Databases/cosmos/rspb_gola_20190409_embedded.json') as f:
    gola_embedded = json.load(f)

In [26]:
len(gola_embedded)

62835

In [27]:
sample(gola_embedded, 1)

[{'location': 'P0440__C11',
  'file_name': 'P0440/C11/P0440__C11__2012-05-26__10-16-28(6).JPG',
  'datetime': '26/05/2012 10:16',
  'image_id': 'P0440__C11__2012-05-26__10-16-28(6)',
  'dataset': 'rspb_gola',
  'annotations': {'species': ['empty']},
  'height': 1536,
  'width': 2048}]

Add sequence info to the RSPB database using info in the file names. 

In [28]:
for e in gola_embedded:
    frame_num = int(e['image_id'].split('(')[1].split(')')[0])
    seq_id = e['image_id'].split('(')[0]
    e['seq_id'] = seq_id
    e['frame_num'] = frame_num

In [29]:
for e in gola_embedded:
    if 'species' not in e['annotations']:
        e['annotations']['species'] = ['__label_unavailable']

## wcs_20190817

In [5]:
with open('/Users/siyuyang/OneDrive - Microsoft/AI4Earth/CameraTrap/Databases/cosmos/wcs_20190817_embedded.json') as f:
    wcs_embedded = json.load(f)

In [6]:
len(wcs_embedded)

1207490

In [7]:
sample(wcs_embedded, 1)

[{'datetime': '2008-02-19 13:09:05.000',
  'seq_id': 'ken-007-d0037-1040',
  'seq_num_frames': 33,
  'frame_num': 2,
  'match_level': 1,
  'file_name': 'animals/0200/0530.jpg',
  'width': 1920,
  'image_id': '0fb7c9ee-92d5-11e9-a2bf-000d3a74c7de',
  'location': '1256',
  'corrupt': False,
  'dataset': 'wcs',
  'annotations': {'sex': 'unknown',
   'species': ['empty'],
   'count': -1,
   'age': 'unknown'},
  'wcs_id': 'ken-007-d0037-i012594',
  'country_code': 'ken',
  'height': 1080}]

In [12]:
wcs_locations = Counter()

for e in wcs_embedded:
    wcs_locations.update([e['location']])

In [15]:
wcs_locations['unknown']

2191

In [16]:
seqs_unknown = []

for e in wcs_embedded:
    if e['seq_id'] == 'unknown':
        seqs_unknown.append(e)

In [17]:
len(seqs_unknown)

242053

In [20]:
seqs_unknown[-1]

{'datetime': '2013-07-12 04:07:43.000',
 'seq_id': 'unknown',
 'seq_num_frames': -1,
 'frame_num': -1,
 'match_level': 4,
 'file_name': 'animals/0560/0614.jpg',
 'width': 1920,
 'image_id': '9475a0e7-92d5-11e9-9bd6-000d3a74c7de',
 'location': '4342',
 'corrupt': False,
 'dataset': 'wcs',
 'annotations': {'sex': 'male',
  'species': ['leopardus pardalis'],
  'count': 1,
  'age': 'unknown'},
 'wcs_id': 'gtm-003-d0045-i069662',
 'country_code': 'gtm',
 'height': 1080}

Delete the sequence properties for these sequences so that dummy seq ID can be created for them 

In [21]:
for e in wcs_embedded:
    if e['seq_id'] == 'unknown':
        del e['seq_id']
        del e['seq_num_frames']
        del e['frame_num']

In [38]:
corrupt_ims = []
for e in wcs_embedded:
    if e['corrupt']:
        corrupt_ims.append(e)
        
len(corrupt_ims)

78

In [42]:
non_jpg = []
for e in wcs_embedded:
    if not e['file_name'].endswith('.jpg'):
        non_jpg.append(e)
len(non_jpg)

141

Added `.bmp`, `.avi`, `.tif` as allowed endings for `file` in the schema

Some datetime are nan - make these into string so the schema check can pass

In [35]:
for e in wcs_embedded:
    if type(e['datetime']) is not str:
        e['datetime'] = str(e['datetime'])

In [51]:
weird_class_label = []
for e in wcs_embedded:
    if '#ref!' in e['annotations']['species']:
        weird_class_label.append(e)
len(weird_class_label)

38

These appear to be cars

In [52]:
for e in wcs_embedded:
    if '#ref!' in e['annotations']['species']:
        e['annotations']['species'] = ['vehicle']

## wcs_private

In [60]:
with open('/Users/siyuyang/OneDrive - Microsoft/AI4Earth/CameraTrap/Databases/cosmos/wcs_private_20190819_embedded.json') as f:
    wcs_private_embedded = json.load(f)

In [61]:
len(wcs_private_embedded)

162501

In [63]:
for e in wcs_private_embedded:
    if e['seq_id'] == 'unknown':
        del e['seq_id']
        del e['seq_num_frames']
        del e['frame_num']

In [66]:
for e in wcs_private_embedded:
    if type(e['datetime']) is not str:
        e['datetime'] = str(e['datetime'])

## wiitigers

In [5]:
with open('/Users/siyuyang/OneDrive - Microsoft/AI4Earth/CameraTrap/Databases/cosmos/wiitigers_20190815_embedded.json') as f:
    wii_embedded = json.load(f)

In [6]:
len(wii_embedded)
sample(wii_embedded, 1)

1125938

[{'annotations': {'species': ['human']},
  'file_name': 'raw_camtrap/TATR DATA_2014,15,16,17/2018/block_1/73/A/tourist/I__01068.JPG',
  'width': 2576,
  'dataset': 'wiitigers',
  'height': 1984,
  'image_id': 'raw_camtrap~TATR DATA_2014,15,16,17~2018~block_1~73~A~tourist~I__01068'}]

In [15]:
# manually fixed this oen image
for i in wii_embedded:
    if not i['file_name'].endswith(('.JPG', '.jpg')):
        i['file_name'] = i['file_name'].split('.JFG')[0] + '.JPG'

**The following datasets that were annotated in batch 10 did not have their embedded CCT compiled... Some do not yet have a database json.**

## trailguard_night_mara_190515

In [7]:
dataset_name = 'trailguard_night_mara_190515'
annotation_path = anno_batch_10

In [8]:
image_filename_to_bboxes = extract_annotations(annotation_path, dataset_name)

  4%|▎         | 6560/177122 [00:00<00:05, 32792.29it/s]

19 files found in directory at annotation_path
Number of annotation entries found: 177122


100%|██████████| 177122/177122 [00:05<00:00, 31801.70it/s]


54 boxes on 63 images were in the annotation file(s). 215715 boxes skipped because they are not for the requested dataset

Category counts for the bboxes:
animal: 1
person: 53


In [13]:
# checked that this label should be person, not animal

image_filename_to_bboxes['tgv3_01003.jpg'][0]['category'] = 'person'

In [18]:
sequences = []

for image_filename, bbox in image_filename_to_bboxes.items():
    im_cat = 'person' if len(bbox) > 0 else 'empty'
    sequences.append({
        'dataset': 'trailguard_night_mara_190515',
        'seq_id': 'dummy_' + image_filename.split('.jpg')[0],
        'images': [
            {
                'file': image_filename,  # all flat, no separator /
                'class': [im_cat]
            }
        ]
    })

In [20]:
sequences, images_updated = add_annotations_to_sequences(sequences, image_filename_to_bboxes)

Dataset to which the sequences belong to: trailguard_night_mara_190515. The bboxes should also be for this set.
63 images updated; 0 images had their bbox overwritten; 0 images not updated


In [22]:
sequences_schema_check.sequences_schema_check(sequences)

Verified that the sequence items meet requirements not captured by the schema.
Verified that the sequence items conform to the schema.


In [23]:
write_json('/Users/siyuyang/OneDrive - Microsoft/AI4Earth/CameraTrap/Databases/megadb_batches_9_10_11/trailguard_night_mara_190515_w_batch_10_boxes.json', sequences)

## islandconservation_190705

In [5]:
dataset_name = 'islandconservation_190705'
annotation_path = anno_batch_10

In [6]:
image_filename_to_bboxes = extract_annotations(annotation_path, dataset_name)

  2%|▏         | 2901/177122 [00:00<00:12, 14410.80it/s]

19 files found in directory at annotation_path
Number of annotation entries found: 177122


100%|██████████| 177122/177122 [00:08<00:00, 21980.72it/s]

10806 boxes on 30000 images were in the annotation file(s). 204963 boxes skipped because they are not for the requested dataset

Category counts for the bboxes:
animal: 8695
person: 2102
vehicle: 9


In [8]:
9 + 2102 + 8695  # about two-thirds were empty

10806

In [37]:
sequences = []
count = 0
for image_filename, bbox in image_filename_to_bboxes.items():
    im_cat = 'empty' if len(bbox) == 0 else bbox[0]['category']
    if im_cat == 'animal':
        im_cat = 'unidentified'
    sequences.append({
        'dataset': 'islandconservation_190705',
        'seq_id': 'dummy_' + dataset_name + '_' + str(count),
        'images': [
            {
                'file': image_filename.replace('~', '/'),
                'class': [im_cat]
            }
        ]
    })
    
    count += 1

In [38]:
sequences, images_updated = add_annotations_to_sequences(sequences, image_filename_to_bboxes)

Dataset to which the sequences belong to: islandconservation_190705. The bboxes should also be for this set.
30000 images updated; 0 images had their bbox overwritten; 0 images not updated


In [39]:
sample(sequences, 1)

[{'dataset': 'islandconservation_190705',
  'seq_id': 'dummy_islandconservation_190705_1268',
  'images': [{'file': 'Ngeruktabel Camera Study/Cam07a/CAM07A083118/Folder 1/RCNX0022.JPG',
    'class': ['person'],
    'bbox': [{'category': 'person',
      'bbox': [0.08307, 0.02226, 0.915, 0.791]}]}]}]

In [40]:
sequences_schema_check.sequences_schema_check(sequences)

Verified that the sequence items meet requirements not captured by the schema.
Verified that the sequence items conform to the schema.


In [41]:
write_json('/Users/siyuyang/OneDrive - Microsoft/AI4Earth/CameraTrap/Databases/megadb_batches_9_10_11/islandconservation_190705_w_batch_10_boxes.json', sequences)

## parkscanada_20190715

In [5]:
dataset_name = 'parkscanada_20190715'
annotation_path = anno_batch_10

In [6]:
image_filename_to_bboxes = extract_annotations(annotation_path, dataset_name)

  4%|▍         | 6775/177122 [00:00<00:05, 33866.19it/s]

19 files found in directory at annotation_path
Number of annotation entries found: 177122


100%|██████████| 177122/177122 [00:05<00:00, 30678.99it/s]


22 boxes on 22 images were in the annotation file(s). 215747 boxes skipped because they are not for the requested dataset

Category counts for the bboxes:
person: 5
vehicle: 17


In [8]:
sequences = []
count = 0
for image_filename, bbox in image_filename_to_bboxes.items():
    im_cat = 'empty' if len(bbox) == 0 else bbox[0]['category']
    if im_cat == 'animal':
        im_cat = 'unidentified'
    sequences.append({
        'dataset': 'parkscanada_20190715',
        'seq_id': 'dummy_' + 'parkscanada_20190715_cars' + '_' + str(count),
        'images': [
            {
                'file': image_filename.replace('~', '/'),
                'class': [im_cat]
            }
        ]
    })
    
    count += 1

In [10]:
sequences, images_updated = add_annotations_to_sequences(sequences, image_filename_to_bboxes)

Dataset to which the sequences belong to: parkscanada_20190715. The bboxes should also be for this set.
22 images updated; 0 images had their bbox overwritten; 0 images not updated


In [11]:
sequences_schema_check.sequences_schema_check(sequences)

Verified that the sequence items meet requirements not captured by the schema.
Verified that the sequence items conform to the schema.


In [12]:
write_json('/Users/siyuyang/OneDrive - Microsoft/AI4Earth/CameraTrap/Databases/megadb_batches_9_10_11/parkscanada_20190715_w_batch_10_boxes.json', sequences)

## bnf_20190624and0815

In [5]:
with open('/Users/siyuyang/Source/temp_data/CameraTrap/batch_9_10_11/BNF/BNF_20190624and0815_20190826.json') as f:
    bnf_cct = json.load(f)

In [7]:
bnf_embedded = make_cct_embedded(bnf_cct)

Loading image DB...
Number of items from the image DB: 25000
Number of images with more than 1 species: 0 (0.0% of image DB)
No bbox DB provided.


The BNF data is too messy to put into sequences or locations.

In [18]:
len(bnf_embedded)  # only the ones sampled to get bboxes were labeled

25000

In [19]:
for e in bnf_embedded:
    e['image_id'] = e['id']
    del e['id']
    
    if 'annotations' not in e or 'species' not in e['annotations']:
        e['annotations'] = {'species': ['__label_unavailable']}

In [21]:
sample(bnf_embedded, 2)

[{'file_name': '20190624cameratraps/images/OUTROP CAMERA TRAPS ALL/Camera traps ALL historical data/CAMERA TRAPS 2011/Camera trap locations 2011/KM 4 x Railway/Km4 Railway 19-11-11/Cam 2011-5/CDY_0071.JPG',
  'image_id': 15551,
  'annotations': {'species': ['__label_unavailable']}},
 {'file_name': '20190624cameratraps/images/OUTROP CAMERA TRAPS ALL/Camera traps ALL historical data/CAMERA TRAPS 2010/Best of 2010/Best of Agustus/Km 3 Macaque 12-07-2010 (3).JPG',
  'annotations': {'species': ['macaque']},
  'image_id': 14204}]

# Make into MegaDB format

In [22]:
docs = bnf_embedded

In [27]:
dataset_name = 'bnf_20190624and0815'
partial_mega_db_path = '/Users/siyuyang/OneDrive - Microsoft/AI4Earth/CameraTrap/Databases/megadb/bnf_20190624and0815.json'
annotation_path = anno_batch_11
mega_db_with_bbox = '/Users/siyuyang/OneDrive - Microsoft/AI4Earth/CameraTrap/Databases/megadb_batches_9_10_11/bnf_20190624and0815_w_batch_10_boxes.json'

In [28]:
sequences = process_sequences(docs, dataset_name)

The dataset_name is set to bnf_20190624and0815. Please make sure this is correct!
Making a deep copy of docs...
Putting 25000 images into sequences...
Number of sequences: 25000
Checking the location field...

all_img_properties
{'image_id', 'file', 'class'}

img_level_properties
{'image_id', 'file'}

image-level properties that really should be sequence-level
{'class'}

Finished processing sequences.
Example sequence items:

{'seq_id': 'dummy_8422080bbe14412ebfdb078f149bd5c4', 'dataset': 'bnf_20190624and0815', 'images': [{'image_id': 1907, 'file': '20190624cameratraps/images/OUTROP CAMERA TRAPS ALL/CAMERA TRAPS SABANGAU/CAMERA TRAPS 2008 up to date/KM4 X RAILWAY/Km4 x Railway 2008_11_07/squirrelly, Nov 2008 Km 4 x Railway b.JPG'}], 'class': ['squirrel']}

[{'seq_id': 'dummy_501f4568342943d684d46cf46c016d21', 'dataset': 'bnf_20190624and0815', 'images': [{'image_id': 1035, 'file': '20190624cameratraps/images/Kutai 1/Sambar Deer/Cdy00019.JPG'}], 'class': ['sambar deer']}]



In [29]:
sequences_schema_check.sequences_schema_check(sequences)

Verified that the sequence items meet requirements not captured by the schema.
Verified that the sequence items conform to the schema.


In [35]:
# write_json(partial_mega_db_path, sequences)

# Add bounding boxes

In [30]:
image_filename_to_bboxes = extract_annotations(annotation_path, dataset_name)

  4%|▍         | 963/25000 [00:00<00:04, 4808.24it/s]

3 files found in directory at annotation_path
Number of annotation entries found: 25000


100%|██████████| 25000/25000 [00:04<00:00, 5046.70it/s]

21796 boxes on 25000 images were in the annotation file(s). 0 boxes skipped because they are not for the requested dataset

Category counts for the bboxes:
animal: 18513
person: 3281
vehicle: 2


In [34]:
sequences, images_updated = add_annotations_to_sequences(sequences, image_filename_to_bboxes, im_id_map_func=bnf_image_filename_map_func)

Dataset to which the sequences belong to: bnf_20190624and0815. The bboxes should also be for this set.
25000 images updated; 0 images had their bbox overwritten; 0 images not updated


In [35]:
sample(sequences, 2)

[OrderedDict([('dataset', 'bnf_20190624and0815'),
              ('seq_id', 'dummy_a6f837ef549046de98ea53248b47f852'),
              ('images',
               [{'image_id': 7346,
                 'file': '20190624cameratraps/images/OUTROP CAMERA TRAPS ALL/CAMERA TRAPS SABANGAU/CAMERA TRAPS 2012/Camera Trap Locations 2012/T0 x TC/T0 x TC 09-05-12/Cam N1/T0 x TC Pig-t-Macaqus (13).JPG',
                 'bbox': [{'category': 'animal',
                   'bbox': [0.5659, 0.2849, 0.1826, 0.2559]}]}]),
              ('class', ['pig'])]),
 OrderedDict([('dataset', 'bnf_20190624and0815'),
              ('seq_id', 'dummy_085bbb92f167441ea390bfde8d902896'),
              ('images',
               [{'image_id': 7146,
                 'file': '20190624cameratraps/images/OUTROP CAMERA TRAPS ALL/CAMERA TRAPS SABANGAU/CAMERA TRAPS 2012/Camera Trap Locations 2012/T 1A x TD/T 1A x TD 17-07-2012/CAm 2011-14/T 1A x TD Sun Bear.JPG',
                 'bbox': [{'category': 'animal',
                   'bbo

In [36]:
write_json(mega_db_with_bbox, sequences)